<a href="https://colab.research.google.com/github/Dusk707/InfosecCodePrompt/blob/master/InfosecCodePrompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Challenge: Write a selenium based functional test that tests the registration form for creating an account at
this site http://automationpractice.com/index.php.

Test Case Steps:

1. Attempt to create account with no email, expected error message
2. Attempt to create account with invalid email, expected failed field validation and error message on submission
3. Attempt to create account with valid email, expect to proceed to next page upon submission, verify provided email is in the info email field
4. Attempt to register with no required fields entered, expected error message
5. Attempt to register with some but not all required fields filled, expect error message
6. Attempt to register with some non-required fields filled and no required fields, expect error message
7. Attempt to enter a number into first name field then click out of the field, expect failed field validation
8. Attempt to enter a string with special characters (/\[](){}@&$) into first name field then click out of the field, expect failed field validation
9. Attempt to enter a long string into first name field then click out of the field, expect successful field validation
10. Attempt to enter a normal length string into first name field then click out of the field, expect successful field validation and verify this value is auto filled into the address first name field
11. Attempt to enter a string into the last name field, verify this value is auto filled into the address last name field
12. Attempt to enter a string of fewer than five characters into the password field then click out of the field, expect failed field validation
13. Attempt to enter a string of five or more characters containing any of the above from 7-9 into the password field then click out of the field, expect successful field validation
14. Attempt to enter a non-mumeric string into the ZipCode, expect error upon submitting
15. Attempt to enter a numeric string of under 5 characters, expect error upon submit
16. Attempt to enter a numeric string of over 5 characters, expect error upon submit
17. Attempt to enter a numeric string of exactly 5 characters, expect no zipcode error upon submit
18. Attempt to enter a non-numeric string into the mobile phone number field, expect phone number error
19. Attempt to enter a numeric string into the mobile phone number field, expect no error for phone number
20. Attempt to complete form with all required fields properly filled in, expect no errors, taken to the My Account Page

Notes about site functionality:

* Every field in the Your Address section has no field based validation like in the Your Personal Information section
* Can enter all numbers in City field
* Can enter any number of digits into phone number field
* Inconsistent naming of ids and classes (some are camelCase, some use an underscore)

Future improvements to test:
* Proper wait conditions
* Visual debugger integration
* More comprehensive validation functions
* Creatation of getter/setter for fields

Change Log:
* August 27, 2020 - File creatation and test case steps written
* August 28, 2020 - First ten steps of test written, added notes about the test site's functionality and structure
* August 29, 2020 - Final ten steps of test written, added section on future improvments to the test
* August 30, 2020 - Create page objects for test use
* August 31, 2020 - Creates version 2 of test using page objects


In [ ]:
# installs
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# set up driver
import sys
import unittest
import random
import string
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [ ]:
# define commonly functions
def HasElementById(id):
  elems = driver.find_elements_by_id(id)
  return len(elems) > 0

def HasElementByClass(className):
  elems = driver.find_elements_by_class_name(className)
  return len(elems) > 0

def Validation(condition, status, message):
  if status:
    if not condition:
      print(message)
  else:
    if condition:
      print(message)

def GetRandomString(length):
  letters = string.ascii_lowercase
  resultStr = ''.join(random.choice(letters) for i in range(length))
  return resultStr

def ClickButtonByClass(className):
  driver.find_element_by_class_name(className).click()

def ClickButtonById(id):
  driver.find_element_by_id(id).click()

# define page objects
class BasePage(object):
  def __init__(self, driver, base_url=driver.current_url):
        self.base_url = base_url
        self.driver = driver
        self.timeout = 30

class StorefrontPage(BasePage):
  def __init__(self, driver):
    self.driver = driver

  def ClickSignInButton(self):
    ClickButtonByClass("login")
    return SignInPage(driver)

  def ClickSignOutButton(self):
    ClickButtonByClass("logout")

class SignInPage(BasePage):
  def __init__(self, driver):
    self.driver = driver
  
  def ClickCreateAccountButton(self):
     ClickButtonById("SubmitCreate")
     return CreateAccountPage(driver)

  def SetCreateAccountEmail(self, email):
    driver.find_element_by_id("email_create").clear()
    driver.find_element_by_id("email_create").send_keys(email)
    driver.find_element_by_id("email_create").send_keys(Keys.TAB)      

class CreateAccountPage(BasePage):
  def __init__(self, driver):
    self.driver = driver

  def ClearAddressLine1(self):
    driver.find_element_by_id("address1").clear()

  def ClearCity(self):
    driver.find_element_by_id("city").clear()

  def ClearCompany(self):
    driver.find_element_by_id("company").clear()

  def ClearInfoFirstName(self):
    driver.find_element_by_id("customer_firstname").clear()

  def ClearInfoLastName(self):
    driver.find_element_by_id("customer_lastname").clear()

  def ClearMobilePhone(self):
    driver.find_element_by_id("phone_mobile").clear()

  def ClearPassword(self):
    driver.find_element_by_id("passwd").clear()

  def ClearPostalCode(self):
    driver.find_element_by_id("postcode").clear()

  def ClickRegisterButton(self):
    ClickButtonById("submitAccount")
    return MyAccountPage(self.driver)

  def GetErrorMessageText(self):
    return driver.find_element_by_class_name("alert-danger").text

  def GetFieldValueById(self, id):
    return driver.find_element_by_id(id).get_attribute("value")

  def SetAddressLine1(self, address1):
    driver.find_element_by_id("address1").clear()
    driver.find_element_by_id("address1").send_keys(address1)
    driver.find_element_by_id("address1").send_keys(Keys.TAB)

  def SetCity(self, city):
    driver.find_element_by_id("city").clear()
    driver.find_element_by_id("city").send_keys(city)
    driver.find_element_by_id("city").send_keys(Keys.TAB)

  def SetCompany(self, company):
    driver.find_element_by_id("company").clear()
    driver.find_element_by_id("company").send_keys(company)
    driver.find_element_by_id("company").send_keys(Keys.TAB)

  def SetInfoFirstName(self, fName):
    driver.find_element_by_id("customer_firstname").clear()
    driver.find_element_by_id("customer_firstname").send_keys(fName)
    driver.find_element_by_id("customer_firstname").send_keys(Keys.TAB)

  def SetInfoLastName(self, lName):
    driver.find_element_by_id("customer_lastname").clear()
    driver.find_element_by_id("customer_lastname").send_keys(lName)
    driver.find_element_by_id("customer_lastname").send_keys(Keys.TAB)

  def SetMobilePhone(self, mPhone):
    driver.find_element_by_id("phone_mobile").clear()
    driver.find_element_by_id("phone_mobile").send_keys(mPhone)
    driver.find_element_by_id("phone_mobile").send_keys(Keys.TAB)

  def SetPassword(self, password):
    driver.find_element_by_id("passwd").clear()
    driver.find_element_by_id("passwd").send_keys(password)
    driver.find_element_by_id("passwd").send_keys(Keys.TAB)

  def SetPostalCode(self, postalCode):
    driver.find_element_by_id("postcode").clear()
    driver.find_element_by_id("postcode").send_keys(postalCode)
    driver.find_element_by_id("postcode").send_keys(Keys.TAB)

  def SetState(self, stateValue):
    driver.find_element_by_id("id_state").click()
    driver.find_element_by_xpath(f"//*[@id='id_state']/option[{stateValue}]").click()

class MyAccountPage(BasePage):
  def __init__(self, driver):
    self.driver = driver

  def ClickLogOutButton(self):
    ClickButtonByClass("logout")

In [ ]:
# start test
# navigate to registration page to begin test
driver.get("http://automationpractice.com/index.php")
if HasElementByClass("logout"): # if an account is already logged in, log out
  driver.find_element_by_id("logout").click()
  driver.implicitly_wait(10)

driver.find_element_by_class_name("login").click()
driver.implicitly_wait(10)

print("Beginning step 1") # attempt to register without providing email
driver.find_element_by_id("SubmitCreate").click()
Validation(HasElementById("create_account_error"), True, "Error message not displayed")

print("Beginning step 2") # attempt to register with invalid email
emailField = driver.find_element_by_id("email_create")
emailField.clear()
emailField.send_keys("d;ifuagsdfhbasd")
emailField.send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Email field validation is not failing")
driver.find_element_by_id("SubmitCreate").click()
Validation(HasElementById("create_account_error"), True, "Error message not displayed")
    
print("Beginning step 3") # attempt to register with valid email
randString = GetRandomString(8)
email = f"test+{randString}@testing.com"
emailField.clear()
emailField.send_keys(email)
driver.find_element_by_id("SubmitCreate").click()
Validation(HasElementByClass("form-error"), False, "Email field validation is failing")
Validation(HasElementById("create_account_error"), False, "Error message is displayed")
autoFilledEmail = driver.find_element_by_id("email").get_attribute("value")
Validation(email == autoFilledEmail, True, "Auto filled email does not match provided email")
driver.implicitly_wait(10)
    
print("Beginning step 4") # attempt to register with no required fields entered
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 5") # attempt to register with some but not all required fields entered
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys("Bob")
driver.find_element_by_id("customer_lastname").clear()
driver.find_element_by_id("customer_lastname").send_keys("Billing")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 6") # attempt to register with some non-required fields and no required fields entered
driver.find_element_by_id("company").clear()
driver.find_element_by_id("company").send_keys("Blue Tech")
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_lastname").clear()
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 7") # attempt to enter a number into first name field then click out of the field
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys("4")
driver.find_element_by_id("customer_firstname").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 8") # attempt to enter a string with special characters (/[](){}@&$) into first name field then click out of the field
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys("asda/[](){}@&$")
driver.find_element_by_id("customer_firstname").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 9") # attempt to enter a long string into first name field then click out of the field
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys(GetRandomString(50))
driver.find_element_by_id("customer_firstname").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 10") # attempt to enter a normal length string into first name field then click out of the field
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys("Bob")
driver.find_element_by_id("customer_firstname").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
autoFilledFirstName = driver.find_element_by_id("firstname").get_attribute("value")
Validation("Bob" == autoFilledFirstName, True, "Auto filled first name does not match provided first name")
    
print("Beginning step 11") # attempt to enter a string into the last name field
driver.find_element_by_id("customer_lastname").clear()
driver.find_element_by_id("customer_lastname").send_keys("Billings")
driver.implicitly_wait(10)
autoFilledLastName = driver.find_element_by_id("lastname").get_attribute("value")
Validation("Billings" == autoFilledLastName, True, "Auto filled last name does not match provided last name")
    
print("Beginning step 12") # attempt to enter a string of fewer than five characters into the password field then click out of the field
driver.find_element_by_id("passwd").clear()
driver.find_element_by_id("passwd").send_keys("1234")
driver.find_element_by_id("passwd").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), True, "Password field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 13") # attempt to enter a string of five or more characters containing any of the above from 7-9 into the password field then click out of the field
driver.find_element_by_id("passwd").clear()
driver.find_element_by_id("passwd").send_keys("1234[]()")
driver.find_element_by_id("passwd").send_keys(Keys.TAB)
Validation(HasElementByClass("form-error"), False, "Password field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 14") # attempt to enter a non-mumeric string into the ZipCode and submit
driver.find_element_by_id("postcode").clear()
driver.find_element_by_id("postcode").send_keys("asdfg")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("Zip/Postal code" in errorMessage, True, "Error message is missing postal code error")
    
print("Beginning step 15") # attempt to enter a numeric string of under 5 characters and submit
driver.find_element_by_id("postcode").clear()
driver.find_element_by_id("postcode").send_keys("1234")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("Zip/Postal code" in errorMessage, True, "Error message is missing postal code error")
    
print("Beginning step 16") # attempt to enter a numeric string of over 5 characters and submit
driver.find_element_by_id("postcode").clear()
driver.find_element_by_id("postcode").send_keys("1234567")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("Zip/Postal code" in errorMessage, True, "Error message is missing postal code error")
    
print("Beginning step 17") # attempt to enter a numeric string of exactly 5 characters and submit
driver.find_element_by_id("postcode").clear()
driver.find_element_by_id("postcode").send_keys("12345")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("Zip/Postal code" in errorMessage, False, "Error message is displaying postal code error")
    
print("Beginning step 18") # attempt to enter a non-numeric string into the mobile phone number field and submit
driver.find_element_by_id("phone_mobile").clear()
driver.find_element_by_id("phone_mobile").send_keys("asdfg")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("phone_mobile" in errorMessage, True, "Error message is missing mobile phone error")
    
print("Beginning step 19") # attempt to enter a numeric string into the mobile phone number field and submit
driver.find_element_by_id("phone_mobile").clear()
driver.find_element_by_id("phone_mobile").send_keys("5555555555")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = driver.find_element_by_class_name("alert-danger").text
Validation("phone_mobile" in errorMessage, False, "Error message is missing mobile phone error")
    
print("Beginning step 20") # attempt to complete form with all required fields properly filled in
driver.find_element_by_id("customer_firstname").clear()
driver.find_element_by_id("customer_firstname").send_keys("Bob")
driver.find_element_by_id("customer_lastname").clear()
driver.find_element_by_id("customer_lastname").send_keys("Billings")
driver.find_element_by_id("passwd").clear()
driver.find_element_by_id("passwd").send_keys("1234[]()")
driver.find_element_by_id("address1").clear()
driver.find_element_by_id("address1").send_keys("555 5th Street")
driver.find_element_by_id("postcode").clear()
driver.find_element_by_id("city").clear()
driver.find_element_by_id("city").send_keys("Fifth City")
driver.find_element_by_id("id_state").click()
driver.find_element_by_xpath("//*[@id='id_state']/option[7]").click()
driver.find_element_by_id("postcode").send_keys("12345")
driver.find_element_by_id("phone_mobile").clear()
driver.find_element_by_id("phone_mobile").send_keys("5555555555")
driver.find_element_by_id("submitAccount").click()
Validation(HasElementByClass("alert-danger"), False, "Error message is displayed")
Validation(HasElementByClass("info-account"), True, "Successful form submission did not go to expect account page")

# log out after successful run
driver.find_element_by_class_name("logout").click()

Beginning step 1
Beginning step 2
Beginning step 3
Beginning step 4
Beginning step 5
Beginning step 6
Beginning step 7
Beginning step 8
Beginning step 9
Beginning step 10
Beginning step 11
Beginning step 12
Beginning step 13
Beginning step 14
Beginning step 15
Beginning step 16
Beginning step 17
Beginning step 18
Beginning step 19
Beginning step 20


In [ ]:
# start test version 2, using page classes
# navigate to registration page to begin test
driver.get("http://automationpractice.com/index.php")
sfPage = StorefrontPage(driver)
if HasElementByClass("logout"): # if an account is already logged in, log out
  sfPage.ClickSignOutButton()
  driver.implicitly_wait(10)

signInPage = sfPage.ClickSignInButton()
driver.implicitly_wait(10)

print("Beginning step 1") # attempt to register without providing email
signInPage.ClickCreateAccountButton()
Validation(HasElementById("create_account_error"), True, "Error message not displayed")

print("Beginning step 2") # attempt to register with invalid email
signInPage.SetCreateAccountEmail("d;ifuagsdfhbasd")
Validation(HasElementByClass("form-error"), True, "Email field validation is not failing")
signInPage.ClickCreateAccountButton()
Validation(HasElementById("create_account_error"), True, "Error message not displayed")
    
print("Beginning step 3") # attempt to register with valid email
randString = GetRandomString(8)
email = f"test+{randString}@testing.com"
signInPage.SetCreateAccountEmail(email)
createAccountPage = signInPage.ClickCreateAccountButton()
Validation(HasElementByClass("form-error"), False, "Email field validation is failing")
Validation(HasElementById("create_account_error"), False, f"Error message is displayed")
autoFilledEmail = createAccountPage.GetFieldValueById("email")
Validation(email == autoFilledEmail, True, f"Auto filled email of {autoFilledEmail} does not match provided email of {email}")
driver.implicitly_wait(10)

print("Beginning step 4") # attempt to register with no required fields entered
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 5") # attempt to register with some but not all required fields entered
createAccountPage.SetInfoFirstName("Bob")
createAccountPage.SetInfoLastName("Billings")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 6") # attempt to register with some non-required fields and no required fields entered
createAccountPage.SetCompany("Blue Tech")
createAccountPage.ClearInfoFirstName()
createAccountPage.ClearInfoLastName()
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
driver.implicitly_wait(10)
    
print("Beginning step 7") # attempt to enter a number into first name field then click out of the field
createAccountPage.SetInfoFirstName("4")
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 8") # attempt to enter a string with special characters (/[](){}@&$) into first name field then click out of the field
createAccountPage.SetInfoFirstName("asda/[](){}@&$")
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 9") # attempt to enter a long string into first name field then click out of the field
createAccountPage.SetInfoFirstName(GetRandomString(50))
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 10") # attempt to enter a normal length string into first name field then click out of the field
createAccountPage.SetInfoFirstName("Bob")
Validation(HasElementByClass("form-error"), True, "Cusomter first name field validation is not failing")
driver.implicitly_wait(10)
autoFilledFirstName = createAccountPage.GetFieldValueById("firstname")
Validation("Bob" == autoFilledFirstName, True, f"Auto filled first name {autoFilledFirstName} does not match provided first name Bob")
    
print("Beginning step 11") # attempt to enter a string into the last name field
createAccountPage.SetInfoLastName("Billings")
driver.implicitly_wait(10)
autoFilledLastName = createAccountPage.GetFieldValueById("lastname")
Validation("Billings" == autoFilledLastName, True, f"Auto filled last name {autoFilledLastName} does not match provided last name Billings")
    
print("Beginning step 12") # attempt to enter a string of fewer than five characters into the password field then click out of the field
createAccountPage.SetPassword("1234")
Validation(HasElementByClass("form-error"), True, "Password field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 13") # attempt to enter a string of five or more characters containing any of the above from 7-9 into the password field then click out of the field
createAccountPage.SetPassword("1234[]()")
Validation(HasElementByClass("form-error"), False, "Password field validation is not failing")
driver.implicitly_wait(10)
    
print("Beginning step 14") # attempt to enter a non-mumeric string into the ZipCode and submit
createAccountPage.SetPostalCode("asdfg")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("Zip/Postal code" in errorMessage, True, f"Error message is missing postal code error: {errorMessage}")
    
print("Beginning step 15") # attempt to enter a numeric string of under 5 characters and submit
createAccountPage.SetPostalCode("1234")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("Zip/Postal code" in errorMessage, True, f"Error message is missing postal code error: {errorMessage}")
    
print("Beginning step 16") # attempt to enter a numeric string of over 5 characters and submit
createAccountPage.SetPostalCode("1234567")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("Zip/Postal code" in errorMessage, True, f"Error message is missing postal code error: {errorMessage}")
    
print("Beginning step 17") # attempt to enter a numeric string of exactly 5 characters and submit
createAccountPage.SetPostalCode("12345")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("Zip/Postal code" in errorMessage, False, f"Error message is displaying postal code error: {errorMessage}")
    
print("Beginning step 18") # attempt to enter a non-numeric string into the mobile phone number field and submit
createAccountPage.SetMobilePhone("asdfg")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("phone_mobile" in errorMessage, True, f"Error message is missing mobile phone error: {errorMessage}")
    
print("Beginning step 19") # attempt to enter a numeric string into the mobile phone number field and submit
createAccountPage.SetMobilePhone("5555555555")
createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), True, "Error message is not displayed")
errorMessage = createAccountPage.GetErrorMessageText()
Validation("phone_mobile" in errorMessage, False, f"Error message contains mobile phone error: {errorMessage}")
    
print("Beginning step 20") # attempt to complete form with all required fields properly filled in
createAccountPage.SetInfoFirstName("Bob")
createAccountPage.SetInfoLastName("Billings")
createAccountPage.SetPassword("1234[]()")
createAccountPage.SetAddressLine1("555 5th Street")
createAccountPage.SetCity("Fifth City")
createAccountPage.SetState(7)
createAccountPage.SetPostalCode("12345")
createAccountPage.SetMobilePhone("5555555555")
myAccountPage = createAccountPage.ClickRegisterButton()
Validation(HasElementByClass("alert-danger"), False, "Error message is displayed")
Validation(HasElementByClass("info-account"), True, "Successful form submission did not go to expect account page")

# logout for next run
myAccountPage.ClickLogOutButton()

Beginning step 1
Beginning step 2
Beginning step 3
Beginning step 4
Beginning step 5
Beginning step 6
Beginning step 7
Beginning step 8
Beginning step 9
Beginning step 10
Beginning step 11
Beginning step 12
Beginning step 13
Beginning step 14
Beginning step 15
Beginning step 16
Beginning step 17
Beginning step 18
Beginning step 19
Beginning step 20
